# Example notebook

In [1]:
using Random, Statistics, CSV, DataFrames
using LinearAlgebra

In [2]:
include("../impute.jl")
include("../regress.jl")
include("../augment.jl")
include("../count.jl")

size

In [3]:
dataset_list = [d for d in split.(read(`ls ../datasets/`, String), "\n") if length(d) > 0]

32-element Array{SubString{String},1}:
 "arrhythmia"                        
 "auto-mpg"                          
 "automobile"                        
 "breast-cancer-wisconsin-original"  
 "breast-cancer-wisconsin-prognostic"
 "communities-and-crime"             
 "communities-and-crime-2"           
 "credit-approval"                   
 "cylinder-bands"                    
 "dermatology"                       
 "echocardiogram"                    
 "heart-disease-cleveland"           
 "heart-disease-hungarian"           
 ⋮                                   
 "ozone-level-detection-one"         
 "sleep"                             
 "thyroid-disease-allbp"             
 "thyroid-disease-allhyper"          
 "thyroid-disease-allhypo"           
 "thyroid-disease-allrep"            
 "thyroid-disease-dis"               
 "thyroid-disease-sick"              
 "thyroid-disease-sick-euthyroid"    
 "thyroid-disease-thyroid-0387"      
 "trains"                            
 "wiki4he" 

In [4]:
results_table = DataFrame(dataset=[], copynum=[], iter=[], method=[], osr2=[])

,dataset,copynum,iter,method,osr2
,Any,Any,Any,Any,Any


In [5]:
SNR = 4

4

In [ ]:
if !isdir("../results")
    mkdir("../results")
end

In [6]:
dname = "thyroid-disease-thyroid-0387" #dataset_list[1]
i = 1
iter = 1

1

Read in a data file.

In [7]:
X_missing = DataFrame(CSV.read("../datasets/"*dname*"/$i/X_missing.csv"))
X_full = DataFrame(CSV.read("../datasets/"*dname*"/$i/X_full.csv"))
first(X_full, 5)

,N,N_1,N_2,N_3,N_4,N_5,N_6,Test
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Int64
1,29,0.3,1.1,56.0,0.89,62.0,21.0,0
2,29,1.6,1.9,128.0,1.53,90.0,29.0,0
3,41,1.9,2.4,130.0,1.24,103.0,11.0,0
4,36,0.6,2.3,111.0,1.1,101.0,26.0,0
5,32,1.0,2.3,96.0,0.93,103.0,36.0,1


Create output

In [8]:
function softthresholding(x; λ=0.1)
    if x > λ
        return x - λ
    elseif x < -λ
        return x + λ
    else 
        return 0 
    end
end

softthresholding (generic function with 1 method)

In [12]:
Base.size(X_full)

(9172, 8)

In [13]:
n,p = Base.size(X_full)
wtrue = softthresholding.(randn(p))
btrue = rand(1)

test_index = findfirst(names(X_full) .== :Test)
wtrue[test_index] = 0.

μ = mean(Matrix{Float64}(X_full), dims=1)
σ = std(Matrix{Float64}(X_full), dims=1)
σ[findall(σ .== 0)].= 1
X_normalize = (X_full .- μ) ./ σ

Y = Matrix{Float64}(X_normalize)*wtrue .+ btrue

9172-element Array{Float64,1}:
  1.6825665419100178
  1.182137205176931 
  0.817252624381009 
  1.1243548732064799
  1.2653568926203467
  2.1555007713366314
  1.0048312326032487
  0.8171810583231388
  1.316274443893906 
  0.9355427579994737
  0.2651672903182227
  0.559865542426636 
 -0.8175361776595746
  ⋮                 
  1.1393551403719884
  1.3800971704067486
  1.897654648915628 
  1.1744060592854186
  1.4792363746505028
  1.4199307824657252
  1.05795072175138  
  0.6486161641559764
  0.7964916729126427
  1.1705681612129466
  1.932137446896903 
  2.0108838406262235

In [14]:
noise = randn(Base.size(X_full,1))
noise .*= norm(Y)/norm(noise)/SNR
Y .+= noise

9172-element Array{Float64,1}:
  1.7077171783696359 
  1.5169102091480968 
  0.9156845973823136 
  1.0117101990746888 
  1.3556598701845501 
  1.7181096749004787 
  1.5856199089925502 
  0.7536832250573735 
  0.8513588484750612 
  0.4650272620045552 
  0.09313436228949351
  0.69027736664542   
 -0.6670999412557221 
  ⋮                  
  1.4437059274717612 
  1.7300214332307742 
  1.913281559306006  
  0.860474842394087  
  1.2653145854502528 
  2.3672423827453235 
  0.9832808928372327 
  0.9965199531933501 
  0.5613468885753862 
  1.4046009461108413 
  2.140393703511245  
  2.2682599914746633 

Add Offset

X_full[:,:One] = Ref(1.)
X_missing[:,:One] = Ref(1.) ;

- Method 1: Impute missing data using MICE and regress using LASSO.

In [15]:
X_imputed = mice(X_missing);
first(X_imputed, 5)

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = sexp(::Type{RCall.RClass{:list}}, ::DataFrame) at dataframe.jl:25
└ @ RCall /Users/jpauph/.julia/packages/RCall/RPlFw/src/convert/dataframe.jl:25
┌ Warning: RCall.jl: Loading required package: lattice
└ @ RCall /Users/jpauph/.julia/packages/RCall/RPlFw/src/io.jl:110
┌ Warning: RCall.jl: 
│ Attaching package: ‘dplyr’
│ 
│ The following objects are masked from ‘package:stats’:
│ 
│     filter, lag
│ 
│ The following objects are masked from ‘package:base’:
│ 
│     intersect, setdiff, setequal, union
│ 
└ @ RCall /Users/jpauph/.julia/packages/RCall/RPlFw/src/io.jl:110
┌ Warning: RCall.jl: Warning: package ‘bindrcpp’ was built under R version 3.4.4
└ @ RCall /Users/jpauph/.julia/packages/RCall/RPlFw/src/io.jl:110


,N,N_1,N_2,N_3,N_4,N_5,N_6,Test
,Int64,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Int64
1,29,0.3,2.0,95.0,0.9,107.0,22.0,0
2,29,1.6,1.9,128.0,1.08,117.0,27.0,0
3,41,0.6,2.2,92.0,0.8,116.0,11.0,0
4,36,5.3,2.0,89.0,0.72,125.0,26.0,0
5,32,12.0,0.9,60.0,0.74,92.0,36.0,1


In [16]:
linear = regress(Y, X_imputed, lasso=true, alpha=0.8)

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = #regress#3(::Bool, ::Float64, ::Float64, ::Function, ::Array{Float64,1}, ::DataFrame) at regress.jl:27
└ @ Main /Users/jpauph/Dropbox (MIT)/1- Research/PHD/regress.jl:27
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = #regress#3(::Bool, ::Float64, ::Float64, ::Function, ::Array{Float64,1}, ::DataFrame) at regress.jl:28
└ @ Main /Users/jpauph/Dropbox (MIT)/1- Research/PHD/regress.jl:28
┌ Warning: `setindex!(df::DataFrame, v::AbstractVector, col_ind::ColumnIndex)` is deprecated, use `begin
│     df[!, col_ind] = v
│     df
│ end` instead.
│   caller = #regress#3(::Bool, ::Float64, ::Float64, ::Function, ::Array{Float64,1}, ::DataFrame) at regress.jl:39
└ @ Main /Users/jpauph/Dropbox (MIT)/1- Research/PHD/regress.jl:39
┌ Warning: Implicit broadcasting to a new column in DataFrame assignment is deprecated.U

,N,N_1,N_2,N_3,N_4,N_5,N_6,Offset
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,3.77893e-5,-0.0234762,0.375827,-0.0315417,2.23244,0.00606517,0.00029442,0.667828


In [17]:
R2, OSR2 = evaluate(Y, X_imputed, linear)
@show R2
@show OSR2

R2 = 0.7506023714742679
OSR2 = 0.7360859246068208


0.7360859246068208

In [ ]:
push!(results_table, [dname, i, iter, "Impute then regress", OSR2])

- Method 2: Add indicator variables for missingness and impute zeros (finitely adaptive). Regress using LASSO.

In [18]:
X_augmented = hcat(zeroimpute(X_missing), indicatemissing(X_missing, removezerocols=true))
linear2 = regress(Y, X_augmented, lasso=true, alpha=0.8, missing_penalty=2.0)

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = #indicatemissing#4(::Bool, ::Function, ::DataFrame) at augment.jl:27
└ @ Main /Users/jpauph/Dropbox (MIT)/1- Research/PHD/augment.jl:27
┌ Warning: `setindex!(df::DataFrame, v::AbstractVector, ::Colon, col_ind::ColumnIndex)` is deprecated, use `begin
│     df[!, col_ind] = v
│     df
│ end` instead.
│   caller = #indicatemissing#4(::Bool, ::Function, ::DataFrame) at augment.jl:27
└ @ Main /Users/jpauph/Dropbox (MIT)/1- Research/PHD/augment.jl:27


,N,N_1,N_2,N_3,N_4,N_5,N_6,N_1_missing
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,3.99202e-5,-0.0243238,0.461489,-0.0287975,1.9612,0.00365699,0.00477581,-0.0403374


In [19]:
R2, OSR2 = evaluate(Y, X_augmented, linear2)
@show R2
@show OSR2

R2 = 0.7972058988564839
OSR2 = 0.7873012502661293


0.7873012502661293

In [ ]:
push!(results_table, [dname, i, iter, "Augmented", OSR2])

- Method 3: Affinely adaptive

In [20]:
X_affine = augmentaffine(X_missing, removezerocols=true)
linear3 = regress(Y, X_affine, lasso=true, alpha=0.8, missing_penalty=10.0)

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = #augmentaffine#5(::Bool, ::Function, ::DataFrame) at augment.jl:45
└ @ Main /Users/jpauph/Dropbox (MIT)/1- Research/PHD/augment.jl:45
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = #augmentaffine#5(::Bool, ::Function, ::DataFrame) at augment.jl:45
└ @ Main /Users/jpauph/Dropbox (MIT)/1- Research/PHD/augment.jl:45
┌ Warning: `setindex!(df::DataFrame, v::AbstractVector, col_ind::ColumnIndex)` is deprecated, use `begin
│     df[!, col_ind] = v
│     df
│ end` instead.
│   caller = #augmentaffine#5(::Bool, ::Function, ::DataFrame) at augment.jl:45
└ @ Main /Users/jpauph/Dropbox (MIT)/1- Research/PHD/augment.jl:45


,N,N_1,N_2,N_3,N_4,N_5,N_6,N_1_missing
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,3.8973e-5,-0.0241811,0.495174,-0.0289264,1.56968,0.00219125,0.00235848,0.0


In [21]:
R2, OSR2 = evaluate(Y, X_affine, linear3)
@show R2
@show OSR2

R2 = 0.8084333591361579
OSR2 = 0.7989896100866425


0.7989896100866425

In [ ]:
push!(results_table, [dname, i, iter, "Augmented Affine", OSR2])

In [ ]:
CSV.write("../results/"*dname*"_$i.csv", results_table)